# BEFORE OBTAIN THE SDS ...

This code shows how to prepare the needed files and directories obtain the Satellite-Derived-Shorelines.

In [42]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import os
import pandas as pd
import numpy as np
from PRISMA_SDS import PRIS_tools, PRIS_img, PRIS_profiles

import warnings
warnings.filterwarnings("ignore")

import matplotlib
import matplotlib.pyplot as plt

plt.ion()
%matplotlib qt
matplotlib.use("Qt5Agg")


import geopandas as gpd

><ins>***For temporal analysis, it is recommended to georeference the PRISMA images zone/zones of interest to one reference image or with Ground Truth points coordinates if available. This will avoid the errors introduced by possible significant geo-accuracy errors between images.***</ins>

**ENSURE PSYPTOOLS is installed**
___

In [ ]:
# Why I have to install it?
pip install pysptools

###  **1. DEFINE THE DIRECTORIES**

---

In [3]:
path0 = os.getcwd()
dir_scenes = os.path.join(path0, 'SCENES')

try:
    os.path.exists(dir_scenes)
except:
    os.makedirs(dir_scenes)

Generate directories of the countries and beaches to be studied, inside the previously generated SCENES folder:

    * Countries should be named with a 2-letter code (e.g. Francia --> FR)
    * Beaches should be named with a code of 2 letters + 1 number (e.g. Beach 1 in FR, FR01)

In [4]:
# Prepare specific beach files

scene = 'FR' 
beach = 'FR02'

paths = PRIS_tools.generate_structure(path0, scene, beach)

### 2. PREPARE THE IMAGES FOR THE SDS EXTRACTION PROCESS

---

><ins>***IN BOTH CASES, IT IS RECOMMENDED TO CHECK THE IMAGES GEO-ACCURACY, AND CO-REGISTRATE IT IF NECESSARY.***</ins> 


    A. The availble image/images are the original 30*30 meters scenes in he5 format.
       The original PRISMA image/images should be placed inside the country folder with its original name.    

In [5]:
# Transform the PAN image from the he5 format into TIFF

error_matrix = False

PRIS_img.generate_PAN(paths, error_matrix)

The total number of images:  0


In [5]:
# Transform the HS cube from the he5 format into TIFF

error_matrix = False
save_rgb = False

PRIS_img.generate_HS(paths, error_matrix, save_rgb)

The total number of images:  1


In [48]:
# Define the square parameters, which defines the image area of interest

coords_upper_left = [4700, 3657] # column and row in image coordinates of the PAN image (../SCENES/scene/PAN)
square_length = 1000 # in meters
square_height = 300 # in meters

In [49]:
PRIS_img.CropToSquare(coords_upper_left, square_length, square_height, paths, beach)

... working on FR02_20211202103824_HS_sub.
... working on FR02_20211202103824_PAN_sub.


    B. Add the square cropped image.
    
       The hyperspectral component (HS) and the panchromatic (PAN) one in TIFF format should be placed 
       in their correspondent directories:
    
        * Square cropped PAN: ../SCENES/Country(FR)/Beach(FR01)/PAN/Square_Cropped
        * Square cropped HS:  ../SCENES/Country(FR)/Beach(FR01)/HS/Square_Cropped
        
       The components should be named as follows:
       
        * Square cropped PAN: Beach_YYYYmmddHHmmSS_PAN_sub.tiff (FR01_20211202103824_PAN_sub.tiff)
        * Square cropped HS:  Beach_YYYYmmddHHmmSS_HS_sub.tiff (FR01_20211202103824_HS_sub.tiff)
         
    

><font color='darkred'>**With both options, the relation between HS and PAN sizes should meet: <br><br> &emsp; $\frac{PAN_{rows}}{HS_{rows}}$ = 6,    &emsp;      $\frac{PAN_{columns}}{HS_{columns}}$ = 6** <br><br> **This relation is mandatory to perform the image pansharpening.**</font> 

### **3. PANSHARP THE IMAGES**

---

In [50]:
PRIS_img.PanSharpening(paths)

Step 1: Filter application ...
Step 2: Calculating synthetic intensity ...
Step 3: Obtaining coefficients ...
Step 4: Performing fusion ...
Step 5: Reshape the fused matrix ...
A total of 1 panSharpened images have been saved


### **4. CLICK THE BASELINE DELIMITING THE BEACH**

---

Click the terrestial limit of the aerial beach. <br>
The baseline should be located inside the beach and not on the zones behid it.

In [51]:
PRIS_profiles.BaselineGenerator(paths)

### 5. GENERATE THE PROFILES

---

    1. Automatically generate the shore normal profiles indicating their: length and rotation

In [52]:
length = 120  # profiles length, the profiles should be long enougth to 
rotation = 85 # degrees that the profiles will be rotated
sense = 'negative' # + rotates clockwise and - anti

PRIS_profiles.AutomaticProfiles(paths, sense, length, rotation, 0)

The profiles have been generated...


    2. Add the pre-defined profiles in SHP format.
    
        The shapefile should be located in ../SCENES/Country(FR)/Beach(FR01)/QGIS/Profiles and named as:
        Beach_date_rotation_profiles.shp (FR01_20211202103824_85_profiles.shp)

### **6. GENERATE THE BEACH CROPPED PANSHARPENED IMAGES**

---

In [53]:
PRIS_tools.beach_crop_shapefile(paths)
PRIS_img.crop_PanSharp_to_beach(paths)

Creating:  FR02_20211202103824_PANS_BeachArea.tiff


### **7. GENERATE THE IMAGE GRID FILE**

---

In [54]:
PRIS_img.Generate_GridRaster(paths)

Analyzing imgage:  FR02_20211202103824_PANS_sub.tiff


Two folder structures expamples necessary to run the jupyter notebooks are provided inside the folder SCENES